# Assignment - 2: Data Representation and Point Cloud Operations

Team Name: \<team name here\> 

Roll number: \<Roll number here (in sorted order)\>

# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 26/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.

# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

In [14]:
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

import cv2
import open3d as o3d
from glob import glob

# Introduction to types of Transformations and Homogeneous coordinates

In robotics applications, it is inevitable to keep track of the frames of multiple objects/worlds. These frames can be transformations from one coordinate frame to the other. **Homogeneous coordinates** help in keeping track of various coordinate frames and allow performing composition of various transforms. We will first try to understand between types of transformations and their invariant properties.
1. What is the difference between Affine, Similarity, and Euclidean transform? What are the invariant properities of each type of transform?
2. Watch this [video](https://www.youtube.com/watch?v=PvEl63t-opM) to briefly understand homogeneous coordinates. What are points at infinity? What type of transformation can you apply to transform a point from infinity to a point that is not at infinity? 
3. Using homogeneous coordinates we can represent different types of transformation as point transforms vs. frame transforms. Concatenation of transforms (whether you post multiply transformation matrices or pre-multiply transformation matrices) depends on the problem and how you are viewing it. Try to understand the difference between frame vs. point transformations from this [video](https://youtu.be/Za7Sdegf8m8?t=1834). Let's assume that our camera and world frames are coinciding with each other. We need to estimate the camera to world **frame** transformation matrix after applying the transformations defined below in terms of $T_i$.We apply **frame** transform to move the camera in the world in the following order:
    1. $T_1$ from the camera coordinate frame.
    2. $T_2$ from the world coordinate frame.
    3. $T_3$ from the world coordinate frame.
    4. $T_4$ from the camera coordinate frame.
    5. $T_5$ from the camera coordinate frame.


<font color='blue' size=4.0/> 1) What is the difference between Affine, Similarity, and Euclidean transform? What are the invariant properities of each type of transform?

* **Affine Transformations** are of the form $f(x) = Ax + b$, where $A \in \mathbb R^{m \times n}$, $b \in \mathbb R^{m}$ for $x \in \mathbb R^{n}$. $A$ is a linear transformation matrix and $b$ represents the translation vector. Affine transformation preserve the collinearity of points, parallel lines, convexity of sets and ratio of length of parallel line segments. Scaling, Rotation, Translation, Reflection and Shear can all be derived from Affine transformation.
* **Similarity Transformations** have two forms, one for points and another one for frames. 
    * For points, we have $f(x) = rAx + t$, where $A$ is $n \times n$ orthogonal matrix, $r$ is a scalar, $t \in \mathbb R^{n}$ is translation vector and $x \in \mathbb R^{n}$. This is a special case of Affine transformation as matrix $A$ is constrained to be orthogonal to represent rotations. Here, points undergo translation, rotation and scaling. Shapes are preserved under similarity transformation for points.
    * For frames, we have $B = RAR^{-1}$, with $A,B,R \in \mathbb R^{n \times n}$. For our purpose, $n=3$, $A$ is linear transformation in Frame-$0$, $R$ is coordinate transform of Frame-$1$ w.r.t. Frame-$0$ and $B$ is the same linear transformation as $A$ in Frame-$1$. Under this definition, the eigenvalues of $A$ and $B$ are preserved.
* **Eucledian Transformation** has the form $f(x) = Rx + t$, here $R \in \mathbb R^{3 \times 3}$ is orthogonal matrix and $t \in \mathbb R^3$ is translation vector. They are special cases of affine transformation and similarity transformation (with $r=1$). This transformation preserves the shape, size and distance between points.

<font color='blue' size=4.0/> 2) Watch this video to briefly understand homogeneous coordinates. What are points at infinity? What type of transformation can you apply to transform a point from infinity to a point that is not at infinity?

Points at infinity are the points which are very far from the camera. They can be transformerd to points not at infinity by setting the last coordinate in homogenous coordinates to 0.

<font color='blue' size=4.0/> 3) Estimate the camera to world frame transformation matrix 

Transformation about current frame is represented by post-multiplication, whereas transformation about fixed frame is represented by pre-multiplication. Following are the transformation matrices after each step.
- A:  $T_{i}T_1$
- B:  $T_{2}T_{i}T_{1}$
- C:  $T_{3}T_{2}T_{i}T_{1}$
- D:  $T_{3}T_{2}T_{i}T_{1}T_{4}$
- E:  $T_{3}T_{2}T_{i}T_{1}T_{4}T_{5}$

Final transformation matrix is $T_{3}T_{2}T_{i}T_{1}T_{4}T_{5}$

# Visualise the Data

Point clouds are a collection of points that represent a 3D shape or feature. Each point has its own set of X, Y and Z coordinates and in some cases additional attributes. A popular way to obtain this is by photogrammetry, though here we will use LiDAR data.

LiDAR is a remote sensing process which collects measurements used to create 3D models and maps of objects and environments. Using ultraviolet, visible, or near-infrared light, LiDAR gauges spatial relationships and shapes by measuring the time it takes for signals to bounce off objects and return to the scanner.

1. Download the data from [here](https://iiitaphyd-my.sharepoint.com/:f:/g/personal/venkata_surya_students_iiit_ac_in/EnYAMaTVIhJItzKYqtahE30BRKB6p6UfHN3TyJzvo6Mw0g?e=PegWds). It contains the LIDAR sensor output and odometry information per frame.

    The .bin files contain the 3D point cloud captured by the LIDAR in this format - x, y, z, and reflectance. 

    The odometry information is given in the `odometry.txt` file, which is a 12 element vector. Reshape each of the first 77 rows to a 3x4 matrix to obtain the pose.
    

2. Obtain the point cloud from this and visualise for 1-2 frames.

In [59]:
odometry = np.loadtxt("./data/odometry.txt", delimiter=' ')
odometry = odometry[:77].reshape(77, 3, 4)

bin_files = glob("./data/LiDAR/*")
bin_files.sort()

no_frames = len(bin_files)

In [60]:
# VIRIDIS = np.array(cm.get_cmap('plasma').colors)
# VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
# print(VIRIDIS.shape, VID_RANGE.shape)

In [61]:
# pcd = np.fromfile(bin_files[0], dtype=np.float32)
# pcd = pcd.reshape(-1, 4)

# points = pcd[:, :-1]
# points[:, :1] = -points[:, :1]

# intensity = pcd[:, -1]
# intensity_col = 1.0 - np.log(intensity+0.01) / np.log(np.exp(-0.004 * 100))
# int_color = np.c_[
#     np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 0]),
#     np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 1]),
#     np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 2])]

# point_cloud = o3d.geometry.PointCloud()
# point_cloud.points = o3d.utility.Vector3dVector(points)
# point_cloud.colors = o3d.utility.Vector3dVector(int_color)


# axis = o3d.geometry.LineSet()
# axis.points = o3d.utility.Vector3dVector(np.array([
#     [0.0, 0.0, 0.0],
#     [1.0, 0.0, 0.0],
#     [0.0, 1.0, 0.0],
#     [0.0, 0.0, 1.0]]))
# axis.lines = o3d.utility.Vector2iVector(np.array([
#     [0, 1],
#     [0, 2],
#     [0, 3]]))
# axis.colors = o3d.utility.Vector3dVector(np.array([
#     [1.0, 0.0, 0.0],
#     [0.0, 1.0, 0.0],
#     [0.0, 0.0, 1.0]]))

# o3d.visualization.draw([point_cloud, axis])

In [63]:
indices = np.random.choice(list(range(no_frames)), 2)
print(indices)

for index in indices:
    pcd = np.fromfile(bin_files[index], dtype=np.float32)
    pcd = pcd.reshape(-1, 4)

    points = pcd[:, :-1]
    points[:, :1] = -points[:, :1]

    reflectance = pcd[:, -1]
    colors = np.stack([reflectance, reflectance, reflectance], axis=1)

    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    point_cloud.colors = o3d.utility.Vector3dVector(colors)
    
    o3d.visualization.draw(point_cloud)
    
    frame_num += 1

[28 62]


# Transform 

The point cloud obtained is with respect to the LiDAR frame. The poses however, are in the camera frame. If we want to combine the point clouds from various frames, we need to bring them to the camera frame. 

1. Refer to the image below and apply the required transformation to the point cloud. 

2. Then, register all point clouds into a common reference frame and visualise it (Open3D). It is helpful to use homogeneous coordinates to keep track of the different frames.

3. Write a function to transform the registered point cloud from the world to the $i^{th}$ camera frame, wherein $i$ is the input to the function.

4. \[Bonus\] Move around in the registered point cloud using arrow keys like you would do in a game. For this you will have to regularly transform the entire registered world to your current camera frame and visualize repeatedly. You may choose to avoid visualizing points that are behind the camera in this case as they are not visible from the scene. You may also visualize points at a max depth to make the process easier.

![](./img/transform.png)

# Occupancy Map

Occupancy grid maps are discrete fine grain grid maps. These maps can be either 2-D or 3-D. Each cell in the occupancy grid map contains information on the physical objects present in the corresponding space. Since these maps shed light on what parts of the environment are occupied, and what is not, they are really useful for path planning and navigation.

Occupancy grid maps are probabilistic in nature due to noisy measurements. Each cell can have three states: Occupied, unoccupied, and unknown. For the purpose of this assignment, you can ignore the unknown and work in a binary setting where 1 is occupied and 0 is unoccupied.

1. The task here is to create an occupancy map for each LiDAR scan. You do not need to apply bayesian update rules here, just keep it simple. 

2. Now, using the *registered* point cloud, generate occupancy maps for each frame. What difference do you expect to see between the two methods?

You can mark a cell as occupied based on a threshold of how many different z values are there for a particular (x,y) cell.